# Kernel chido 

In [1]:
using CUDA, LinearAlgebra, Statistics

In [2]:
A = [3 -2 0; 1 3 -1; 2 1 4]
b = [-3, -3, 6]

M = Diagonal(A) # Tal vez esta es una mejor forma(?)

N = M - A

T = inv(M) * N

f = inv(M) * b

nT, mT = size(T)

S = zeros(Int64, nT)

[S[i] += 1 for i in 1:nT, j in 1:mT if T[i,j] != 0]

P = zeros(nT, mT)
[P[i,j] = 1/S[i] for i in 1:nT, j in 1:mT if T[i,j] != 0];
#0.0  1.0  0.0
#0.5  0.0  0.5
#0.5  0.5  0.0

[P[i,j] = sum(P[i,1:j]) for i = 1:nT, j = 1:mT if P[i,j] != 0];
#0.0 1.0 0.0
#0.5 0.0 1.0
#0.5 1.0 0.0

In [3]:
T = Float32.(T)

3×3 Array{Float32,2}:
  0.0        0.666667  0.0
 -0.333333   0.0       0.333333
 -0.5       -0.25      0.0

In [4]:
P = Float32.(P)

3×3 Array{Float32,2}:
 0.0  1.0  0.0
 0.5  0.0  1.0
 0.5  1.0  0.0

In [5]:
e = 3

u = rand(e)
Point = [1]
Nextpoint = []
#nextpoint = [u[i] for i = 1:40, while u]
for tamaño = 1:e
    np = 1
    while u[tamaño] >= P[Point[tamaño], np]
        np += 1
    end
    push!(Nextpoint, np)
    push!(Point, np)
end

In [6]:
var = 1 # Aquí cambiamos la posición del vector solución que queremos aproximar, ya sea X[1], X[2] o X[3]
d_W = CUDA.ones(Float32, e) # Aquí suponemos W_0 = 1.
d_point = CuArray(Point)
d_nextpoint = CuArray{Int32}(Nextpoint)
d_W_new = CuArray{Float32}(undef, e)
d_P = CuArray{Float32}(round.(P, digits = 6))
d_T = CuArray{Float32}(round.(T, digits = 6))
d_X = CUDA.fill(Float32(f[var]), e)
d_f = CuArray(Float32.(f));

In [11]:
d_T

3×3 CuArray{Float32,2}:
  0.0        0.666667  0.0
 -0.333333   0.0       0.333333
 -0.5       -0.25      0.0

In [12]:
d_T.^4

3×3 CuArray{Float32,2}:
 0.0        0.197531    0.0
 0.0123456  0.0         0.0123456
 0.0625     0.00390625  0.0

In [16]:
function kernel(W, W_new, X, T, P, nextpoint, point, f)
    i = (blockIdx().x-1) * blockDim().x + threadIdx().x
    #W_new[i] = W[i] * (T[point[i], nextpoint[i]]/P[point[i], nextpoint[i]])
    @cushow T[1,1]^4
    W_new[i] = 0.0f0^14#W[i]*(T[point[i], nextpoint[i]])^4
    #X[i] += W_new[i] * f[nextpoint[i]]
    return
end

kernel (generic function with 1 method)

In [17]:
@cuda threads = e kernel(d_W, d_W_new, d_X, d_T, d_P, d_nextpoint, d_point, d_f)

LLVM.LLVMException: LLVM error: Cannot select: 0x6933878: f32 = fpow 0x6933c20, ConstantFP:f32<4.000000e+00>, math.jl:884 @[ intfuncs.jl:265 @[ /home/oscar/.julia/packages/CUDA/7vLVC/src/device/intrinsics/output.jl:230 @[ In[16]:4 ] ] ]
  0x6933c20: f32,ch = load<(load 4 from %ir.31, !tbaa !128, addrspace 1)> 0x52cc2a8, 0x69330c0, undef:i64, /home/oscar/.julia/packages/LLVM/T8ZBA/src/interop/base.jl:53 @[ /home/oscar/.julia/packages/CUDA/7vLVC/src/device/pointer.jl:115 @[ /home/oscar/.julia/packages/CUDA/7vLVC/src/device/pointer.jl:115 @[ /home/oscar/.julia/packages/CUDA/7vLVC/src/device/pointer.jl:245 @[ /home/oscar/.julia/packages/CUDA/7vLVC/src/device/array.jl:80 @[ /home/oscar/.julia/packages/CUDA/7vLVC/src/device/array.jl:99 @[ abstractarray.jl:1003 @[ abstractarray.jl:980 @[ /home/oscar/.julia/packages/CUDA/7vLVC/src/device/intrinsics/output.jl:230 @[ In[16]:4 ] ] ] ] ] ] ] ] ]
    0x69330c0: i64,ch = CopyFromReg 0x52cc2a8, Register:i64 %0, /home/oscar/.julia/packages/LLVM/T8ZBA/

In [21]:
@which 2^3

^(x::T, p::T) where T<:Integer in Base at intfuncs.jl:238

In [136]:
mean(Array(d_X))

-1.0f0

In [228]:
for i = 1:3
    a = T[Point[i], Nextpoint[i]]/P[Point[i], Nextpoint[i]]
    println(a%1)
end

0.6666666666666666
-0.6666666666666666
0.6666666666666666


In [74]:
point = 2
W_0 = 1
W = W_0
X = zeros(Float32, mT)
for tamaño = 1:20
    u = rand()
    nextpoint = 1
    while u >= P[point, nextpoint]
        nextpoint += 1
    end
    W_new = W *(T[point, nextpoint]/P[point, nextpoint])
    X[1] += W_new * f[nextpoint] # ------------------------Aquí
    W = W_new
    point = nextpoint
end

In [75]:
X[1]

0.5127022f0